In [1]:
import dlt
import pyarrow.parquet as pq
import fsspec

In [2]:
# Define un recurso que lee datos de un parquet remoto
@dlt.resource(table_name="df_data")
def my_df():
    parquet_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet"
    with fsspec.open(parquet_url, mode="rb") as f:
        table = pq.read_table(f)
        yield table

In [3]:
# 🚀 Pipeline configurado con destino filesystem
# Las credenciales se leerán automáticamente desde `.dlt/secrets.toml`
pipeline = dlt.pipeline(
    pipeline_name="parquet_to_minio",
    destination="filesystem",
    dataset_name="taxis_parquet",
)

In [4]:
# Ejecutamos el pipeline
load_info = pipeline.run(
    my_df,
    loader_file_format="parquet",
    write_disposition="replace"
)
print(load_info)

Pipeline parquet_to_minio load step completed in 0.66 seconds
1 load package(s) were loaded to destination filesystem and into dataset taxis_parquet
The filesystem destination used s3://taxis location to store data
Load package 1757200138.9694595 is LOADED and contains no failed jobs
